In [1]:
import ConfigArgs as args
import psycopg2
from tqdm import tqdm

In [2]:
conn = psycopg2.connect(args.conn_str)
cursor = conn.cursor()

In [3]:
sql = """SELECT image_name, triplets FROM "Image2GraphEmbedding";"""
cursor.execute(sql)
records = cursor.fetchall()

In [4]:
def parse_postgres_array(data):
    cleaned_data = data.replace('{', '').replace('}', '').replace('""', '"')
    list_data = cleaned_data.split('","')
    list_data = [item.strip('"') for item in list_data]
    
    return list_data
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    if not union:
        return 0.0  # Tránh chia cho 0
    return len(intersection) / len(union)
def extract_subject_relation_object(trips):
    trip = []
    for item in trips:
        words = item.split()
        t_json = {
            "subject": words[0],
            "relation": ' '.join(words[1:-1]),
            "object": words[-1]
        }
        trip.append(t_json)
    return trip

In [5]:
img_id = []
file_image = []
triplet_set = []
trip_by_im = []
for record in records[:]:
    file_image.append(record[0])
    triplet_set.append(set(parse_postgres_array(record[1])))

In [6]:
# combined = list(zip(triplet_set, file_image))
# similarities = [(filename, jaccard_similarity(setQ, triplet_set), triplet_set) for triplet_set, filename in combined]
# top_k_res = sorted(similarities, key=lambda x: x[1], reverse=True)[:2]
# top_k_res

In [13]:
res = []
combined = list(zip(triplet_set, file_image))
for item in tqdm(triplet_set[:20000]):
    setQ = item
    similarities = [(filename, jaccard_similarity(setQ, triplet_set), triplet_set) for triplet_set, filename in combined]
    top_k_res = sorted(similarities, key=lambda x: x[1], reverse=True)[:2]
    for idx, (i, s, t) in enumerate(top_k_res):
        if(idx == 1 and s >= 0.375):
            t_json = {
                "qe": {
                    "image_id": top_k_res[0][0],
                    "trip": list(top_k_res[0][2]),
                },
                "rev": {
                    "image_id": i,
                    "image_id_qe": top_k_res[0][0],
                    "trip": list(t) 
                    # "score": s,
                    # "trip": extract_subject_relation_object(list(t))
                }
            }
            res.append(t_json)
len(res)

100%|██████████| 20000/20000 [30:23<00:00, 10.97it/s]


In [14]:
len(res)

11949

In [17]:
import json
with open('Rev.json', 'w') as f:
    json.dump(res, f)